In [1]:
import numpy as np 
import matplotlib.pyplot as plt
from matplotlib import gridspec
from ipywidgets import interact, FloatSlider
import sympy as sp

In [2]:
# how to de-dimensionalize the problem?
import sympy as sp
# let the fundamental units be R,V, and G

Rc      =   sp.Matrix([1, 0, 0])
Vc      =   sp.Matrix([0, 1, 0])
G       =   sp.Matrix([0, 0, 1])
M       =   sp.Matrix([1, 2, -1])

B=sp.Matrix(Rc.row_join(Vc).row_join(G).row_join(M))
display(B)

Matrix([
[1, 0, 0,  1],
[0, 1, 0,  2],
[0, 0, 1, -1]])

In [3]:
display(B.nullspace()[0])

Matrix([
[-1],
[-2],
[ 1],
[ 1]])

### non-dimensional number

$ \Pi_1 = \frac{GM_c}{R_c V_c^2}$

which basically tells us, the closer, slower, and more massive we are, the larger the change in velocity. It scales qudratically with velocity, which is cool

$$ \Delta v_ {x} = \frac {2GM(bw^ {2}\cos \alpha + yw_ {\bot }w_ {\parallel}\sin \alpha )}{w((b^ {2}+r_ {s}^ {2})w^ {2}+w_ {\bot}^ {2}y^ {2})} $$

 
$$ \Delta v_ {y}  =- \frac{ 2GMw_{\bot}^ {2} y}{w(( b^ {2} + r_s^ {2} ) y^{2} + y^2 w_{\bot}^ {2}) }$$

<!-- $$ \Delta $ $ v_ {z} $ = $ \frac {2GM(bw^ {2}\sin \alpha -yw_ {\bot }w_ {\parallel}\cos \alpha )}{w((b^ {2}+r_ {s}^ {2})w^ {2}+w_ {\parallel}^ {2}y^ {2})} $$ -->

$$ \Delta v_ {z} = \frac {2GM(bw^ {2}\sin \alpha -yw_ {\bot }w_ {\parallel}\cos \alpha )}{w((b^ {2}+r_ {s}^ {2})w^ {2}+w_ {\parallel}^ {2}y^ {2})} $$



the critical points

$$ \delta v_{y,\text{max}}=\frac{GMw_{\bot}}{w^2\sqrt{b^2+r^2}} $$

$$ y_{fwhm} = \frac{3.5w\sqrt{b^2+r^2}}{w_{\bot}} $$

$$ y_{\text{max prop}} = \frac{w\sqrt{b^2+r^2}}{w_{\bot}} $$

In [4]:
def get_kick_velocity(PI1,M,b,alpha,y,Wperp,Wpar,Rs):
    """ 
    the changes in velocoity of the particle due to the gravitational force of da passage
    """
    Wmag=np.sqrt(Wperp**2+Wpar**2)
    term1 = b*(Wmag**2)*np.cos(alpha)
    term2 = y*Wperp*Wpar*np.sin(alpha)
    term3 = (b**2 + Rs**2)*(Wmag**2)
    term4 = (Wperp**2)*(y**2)

    denominator = Wmag*(term3+term4)

    dX= 2*PI1*M*(term1+term2)/denominator
    dY=-2*PI1*(M*(Wperp**2)*y)/denominator
    # re-arrange the numerator for dZ    
    term1= (b*Wmag**2*np.sin(alpha))
    term2= y*Wperp*Wpar*np.cos(alpha)
    dZ= 2*PI1*M*(term1-term2)/denominator
    return np.array([dX,dY,dZ])

In [5]:
def get_critical_points(PI1,M,b,Wperp,Wpar,Rs):
    """ 
    From Figure 3
    """
    W=np.sqrt(Wperp**2+Wpar**2)
    dVmax=PI1 * M * Wperp / (W**2 * np.sqrt( Rs**2 + b**2))
    y_cord= (W/Wperp)*np.sqrt(Rs**2 + b**2)
    half_width = 3.5 * W * np.sqrt(Rs**2 + b**2) / (Wperp)
    return dVmax, y_cord, half_width

In [6]:
def plot_three_phases(axis,b, alpha, Wperp, Wpar):
    # make vector per impact parameter
    B = np.array([b*np.cos(alpha),0,b*np.sin(alpha)])

    # Define the line for the perturber
    W=np.array([-Wperp*np.sin(alpha),Wpar,Wperp*np.cos(alpha)])
    Wnorm=W/np.linalg.norm(W)
    
    # find where the line intersects the x-y plane for clarity
    t_inter = -B[2]/Wnorm[2]

    # make time array for the line
    t = np.linspace(-t_inter,t_inter,10)
    intersection_point=np.array([Wnorm[0]*t_inter + B[0],Wnorm[1]*t_inter+B[1],Wnorm[2]*t_inter+B[2]])    
    # make a parametric line for W
    Wline = np.array([Wnorm[0]*t + B[0],Wnorm[1]*t+B[1],Wnorm[2]*t+B[2]])

    # find the position of the particle at that time
    
    # Define the parameters for the arc
    radius = np.linalg.norm(B)/2
    theta = np.linspace(0, alpha, 100)

    # Parametric equations for the 3D arc
    x_arc = radius * np.cos(theta)
    y_arc = radius * np.sin(theta) * np.cos(np.arccos(B[1] / np.linalg.norm(B[1:])))
    z_arc = radius * np.sin(theta) * np.sin(np.arccos(B[1] / np.linalg.norm(B[1:])))



    limit=1
    # Plot the X, Y, and Z coordinate axes
    axis.plot3D([0, limit], [0, 0], [0, 0], 'k',)
    axis.plot3D([0, 0], [0, limit], [0, 0], 'k',)
    axis.plot3D([0, 0], [0, 0], [0, limit], 'k',)

    # Add x-y-z axis labels
    axis.text(limit, 0, 0, 'X', color='k')
    axis.text(0, limit, 0, 'Y', color='k')
    axis.text(0, 0, limit, 'Z', color='k')

    # Plot the vector B
    B_handle = axis.quiver(0, 0, 0, B[0], B[1], B[2], color='r', label='B')

    # Add a label for the vector B
    axis.text(B[0], B[1], B[2], 'b', color='r')

    arc_handle = axis.plot(x_arc,y_arc,z_arc)

    axis.text(x_arc.mean(),y_arc.mean(),z_arc.mean(),r'$\alpha$',color='b')

    # plot the line W
    Wline_handle = axis.plot(Wline[0],Wline[1],Wline[2],'b')


    # Calculate the intersection point of the line W with the xy-plane

    # Plot the line from the intersection point to the xy-plane
    axis.plot3D([intersection_point[0], intersection_point[0]], [0, intersection_point[1]], [0, 0], 'k--')

    # Update the plot
    # plt.show()
    
    axis.scatter(intersection_point[0], intersection_point[1], intersection_point[2], color='k')
    # ax.plot([intersection_point[0],intersection_point[0]],[0,0],[0,intersection_point[2]],'k--')


    axis.set_xlim(-limit, limit)
    axis.set_ylim(-limit, limit)
    axis.set_zlim(-limit, limit)
    # Turn off the axis panes
    axis.axis('off')

    # Return the plot handles
    return axis, B_handle, arc_handle, Wline_handle

In [7]:
def update_figure(PI1,M,b,alpha,Wperp,Wpar,Rs,):
    # ylims=(-50,50)
    xlims=(-1,1)
    fig=plt.figure(figsize=(15, 4))
    fig.clf()
    y=np.linspace(-50,50,1000)
    dV=get_kick_velocity(PI1,M,b,alpha,y,Wperp,Wpar,Rs)
    
    # Create a new figure
    # fig = plt.figure(figsize=(15, 4))

    # Add two 3D subplots to the figure using the grid layout
    gs = gridspec.GridSpec(1, 2, width_ratios=[1, 2], wspace=0)
    axes=[]
    axes.append(fig.add_subplot(gs[0]))
    axes.append(fig.add_subplot(gs[1], projection='3d'))

    non_dimen_text = r"$\Pi$: $\frac{{GM_c}}{{V_c^2 R_c}}$ = {:2.1f}".format(PI1)
    axes[1].text(-1.5, 0.5,0, non_dimen_text, fontsize=18, ha='center')

    plot_three_phases(axes[1], b, alpha, Wperp, Wpar)
    
    # get the critical points
    dVmax, y_cord, half_width = get_critical_points(PI1,M,b,Wperp,Wpar,Rs)
    
    # plot the critical points
    markerline, stemlines, baseline=axes[0].stem([y_cord,-y_cord], [-dVmax,dVmax], linefmt='k--', markerfmt='ko', basefmt='k-',)
    plt.setp(stemlines, 'linewidth', 0.5)
    xrange=np.linspace(-half_width/2,half_width/2,2)-2*y_cord
    axes[0].plot(xrange,[dVmax/2,dVmax/2],'k--')
    xrange=np.linspace(-half_width/2,half_width/2,2)+2*y_cord
    axes[0].plot(xrange,[-dVmax/2,-dVmax/2],'k--')
    # axes[0].plot([-(y_cord-half_width),-(y_cord+half_width)],[-dVmax/2,-dVmax/2],'k--')
    axes[0].plot(y, dV[0], label='dVx')
    axes[0].plot(y, dV[1], label='dVy')
    axes[0].plot(y, dV[2], label='dVz')
    axes[0].legend()
    axes[0].set_xlabel('y [$x_c$]')
    axes[0].set_ylabel('dV [$v_c$]')

    title_text=r"b:{:2.2f},$\alpha$:{:2.0f}$^{{\circ}}$, $W_\bot$:{:2.1f} $W_\parallel${:2.1f}".format(b,(180/np.pi)*alpha,Wperp,Wpar)
    axes[0].set_title(title_text)


    # figname="pi_{:2.1f}_b_{:2.2f}_alpha_{:2.0f}_Wperp_{:2.1f}_Wpar_{:2.1f}.png".format(PI1,b,(180/np.pi)*alpha,Wperp,Wpar)
    
    # axes[0].set_xlim(xlims)
    plt.show()
    
    
PI1=2
M=1
b=0.5
Wperp = 2
Wpar = 2
Rs = 1
alpha = np.pi/4

interact(update_figure,
         PI1=FloatSlider(min=1e-2, max=100.0, step=0.1, value=PI1),
         M=FloatSlider(min=0.0, max=10, step=0.1, value=M),
         b=FloatSlider(min=1e-2, max=5.0, step=0.1, value=b),
         alpha=FloatSlider(min=1e-2, max=2*np.pi, step=1/500, value=alpha),
         Wperp=FloatSlider(min=0.0, max=2.0, step=0.1, value=Wperp),
         Wpar=FloatSlider(min=0.0, max=2.0, step=0.1, value=Wpar),
         Rs=FloatSlider(min=1e-2, max=5.0, step=0.1, value=Rs))

interactive(children=(FloatSlider(value=2.0, description='PI1', min=0.01), FloatSlider(value=1.0, description=…

<function __main__.update_figure(PI1, M, b, alpha, Wperp, Wpar, Rs)>

make a set of initial conditions

In [8]:
PI1=2
M=1
b=0.5
Wperp = 2
Wpar = 2
Rs = 1
alpha = np.pi/4

interact(update_figure,
         PI1=FloatSlider(min=1e-2, max=100.0, step=0.1, value=PI1),
         M=FloatSlider(min=0.0, max=10, step=0.1, value=M),
         b=FloatSlider(min=1e-2, max=5.0, step=0.1, value=b),
         alpha=FloatSlider(min=1e-2, max=2*np.pi, step=1/500, value=alpha),
         Wperp=FloatSlider(min=0.0, max=2.0, step=0.1, value=Wperp),
         Wpar=FloatSlider(min=0.0, max=2.0, step=0.1, value=Wpar),
         Rs=FloatSlider(min=1e-2, max=5.0, step=0.1, value=Rs))

interactive(children=(FloatSlider(value=2.0, description='PI1', min=0.01), FloatSlider(value=1.0, description=…

<function __main__.update_figure(PI1, M, b, alpha, Wperp, Wpar, Rs)>

In [9]:
interact(update_figure,
         PI1=FloatSlider(min=1e-2, max=100.0, step=0.1, value=PI1),
         M=FloatSlider(min=0.0, max=10, step=0.1, value=M),
         b=FloatSlider(min=1e-2, max=5.0, step=0.1, value=b),
         alpha=FloatSlider(min=1e-2, max=2*np.pi, step=1/500, value=alpha),
         Wperp=FloatSlider(min=0.0, max=2.0, step=0.1, value=Wperp),
         Wpar=FloatSlider(min=0.0, max=2.0, step=0.1, value=Wpar),
         Rs=FloatSlider(min=1e-2, max=5.0, step=0.1, value=Rs))

interactive(children=(FloatSlider(value=2.0, description='PI1', min=0.01), FloatSlider(value=1.0, description=…

<function __main__.update_figure(PI1, M, b, alpha, Wperp, Wpar, Rs)>

# Range of validity 
The region over which the velocity kick occurs must be much smaller than the orbital radius
$$ \frac{W}{W_\bot} \sqrt{b^2 + r_s^2} <<1 $$

$$ \sqrt{1 + \left(\frac{W_{\parallel}}{W_\bot}\right)^2} \sqrt{b^2 + r_s^2} << r_0 $$

where $r_0$ is the orbital radius. 

The time of the interaction must be very small compared to the orbital time

$$ \frac{\sqrt{b^2 + r_s^2}}{W_\bot} << \frac{r_0}{v_y} $$

## $$ \nabla^2 \left(\phi_\mathrm{disk_0}+\phi_\mathrm{disk_0}+\phi_\mathrm{halo} \right)=4\pi G\left(\rho_\mathrm{disk_0} + \rho_\mathrm{disk_1} + \rho_\mathrm{halo} \right) $$ 

## $$ \vec{a}_{GC,j}  = -\nabla \Phi_{\textrm{MW}} + \sum_{i,i!=j} \frac{GM_i}{\delta |R_{ij}|^{3}}\delta\vec{R}_{ij}$$

## $$ \vec{a}_p  = -\nabla \Phi_{\textrm{MW}} + G M \frac{\vec{r}_{GC}(t)-\vec{r}_p}{\left(b^2 + |\vec{r}_p-\vec{r}_{GC}(t)|^2\right)^{3/2}} + \sum_{i,i!=j} \frac{GM_i}{\delta |R_{ij}|^{3}}\delta\vec{R}_{ij}$$

## $$ n\left(r\right) $$
## $$ N\left(M\right) $$
## $$ b $$
## $$ v_\bot $$
## $$ f\left(v_\bot\right) $$
## $$ M $$
## $$ \mathcal{R}_{\bigcup} (r)$$